In [2]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
dtypes = {'region_code':'object', 'district_code':'object'}

In [31]:
X_train = pd.read_csv('training_set_values.csv',
                      index_col=[0], dtype = dtypes,
                      parse_dates=['date_recorded', 'construction_year'], infer_datetime_format=True)
X_train.head(5)

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   amount_tsh             59400 non-null  float64       
 1   date_recorded          59400 non-null  datetime64[ns]
 2   funder                 55765 non-null  object        
 3   gps_height             59400 non-null  int64         
 4   installer              55745 non-null  object        
 5   longitude              59400 non-null  float64       
 6   latitude               59400 non-null  float64       
 7   wpt_name               59400 non-null  object        
 8   num_private            59400 non-null  int64         
 9   basin                  59400 non-null  object        
 10  subvillage             59029 non-null  object        
 11  region                 59400 non-null  object        
 12  region_code            59400 non-null  object        
 1

In [7]:
y_train = pd.read_csv('training_set_labels.csv')
y_train = y_train.status_group
y_train.head()

0        functional
1        functional
2        functional
3    non functional
4        functional
Name: status_group, dtype: object

In [8]:
X_test = pd.read_csv('test_set_values.csv', index_col=[0],
                    dtype = dtypes)
X_test.info(2)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 50785 to 68707
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             14850 non-null  float64
 1   date_recorded          14850 non-null  object 
 2   funder                 13981 non-null  object 
 3   gps_height             14850 non-null  int64  
 4   installer              13973 non-null  object 
 5   longitude              14850 non-null  float64
 6   latitude               14850 non-null  float64
 7   wpt_name               14850 non-null  object 
 8   num_private            14850 non-null  int64  
 9   basin                  14850 non-null  object 
 10  subvillage             14751 non-null  object 
 11  region                 14850 non-null  object 
 12  region_code            14850 non-null  object 
 13  district_code          14850 non-null  object 
 14  lga                    14850 non-null  object 
 15

In [9]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14850 entries, 50785 to 68707
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             14850 non-null  float64
 1   date_recorded          14850 non-null  object 
 2   funder                 13981 non-null  object 
 3   gps_height             14850 non-null  int64  
 4   installer              13973 non-null  object 
 5   longitude              14850 non-null  float64
 6   latitude               14850 non-null  float64
 7   wpt_name               14850 non-null  object 
 8   num_private            14850 non-null  int64  
 9   basin                  14850 non-null  object 
 10  subvillage             14751 non-null  object 
 11  region                 14850 non-null  object 
 12  region_code            14850 non-null  object 
 13  district_code          14850 non-null  object 
 14  lga                    14850 non-null  object 
 15

## Create Submission Pipeline

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [11]:
columns = ['gps_height']
ct = ColumnTransformer(remainder='drop',
                        transformers=[('select', 'passthrough', columns)])
model_1=Pipeline([
                  ('selector', ct),
                  ('predictor', DecisionTreeClassifier())
])

In [12]:
model_1.fit(X_train, y_train)

Pipeline(steps=[('selector',
                 ColumnTransformer(transformers=[('select', 'passthrough',
                                                  ['gps_height'])])),
                ('predictor', DecisionTreeClassifier())])

In [13]:
model_1.score(X_train, y_train)

0.5714814814814815

In [14]:
date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')

In [15]:
def make_submission(model, X_test):
    y_test_pred = model.predict(X_test)
    predictions = pd.Series(data = y_test_pred,
                            index=X_test.index,
                            name='status_group')
    predictions.to_csv(f'predictions/{date}submission.csv',
                       index=True,
                       header = True)

In [16]:
# make_submission(model_1, X_test)
# This scored a 0.53

# Models With Only Numerical Feautres

In [17]:
# X_train.dtypes

In [18]:
num_feat = X_train.select_dtypes(include='number').columns.to_list()

In [19]:
X_train[['region_code', 'district_code']].head()

,region_code,district_code
id,,
69572,11,5
8776,20,2
34310,21,4
67743,90,63
19728,18,1


In [20]:
X_train.select_dtypes(include='number').describe()

,amount_tsh,gps_height,longitude,latitude,num_private,population
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,179.909983
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,471.482176
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,0.000000
25%,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,0.000000
50%,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,25.000000
75%,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,215.000000
max,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,30500.000000


In [21]:
from sklearn.impute import SimpleImputer

In [22]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=0, strategy='mean'))
])

In [26]:
ct = ColumnTransformer(remainder='drop',
                        transformers=[('numerical', num_pipe, num_feat)])
 

In [27]:
model_2 = Pipeline({
                  ('ct', ct),
                  ('classifier', DecisionTreeClassifier())
})

model_2.fit(X_train, y_train)

model_2.score(X_train, y_train)

0.9841414141414141

In [ ]:
# make_submission(model_2, X_test)
# This got a score of 0.66

# Model with both numerical and categorical features

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [29]:
num_feat

['amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'population']

In [30]:
cat_feat = X_train.select_dtypes(include='object').columns.to_list()
cat_feat

['funder',
 'installer',
 'wpt_name',
 'basin',
 'subvillage',
 'region',
 'region_code',
 'district_code',
 'lga',
 'ward',
 'public_meeting',
 'recorded_by',
 'scheme_management',
 'scheme_name',
 'permit',
 'construction_year',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'quantity_group',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group']

In [34]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=0, strategy='mean')),
    ('scaler', StandardScaler())
])

In [35]:
# Imputer - simple imputer
# Encoder - onehot

cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [36]:
ct = ColumnTransformer(remainder='drop',
                        transformers=[
                            ('numerical', num_pipe, num_feat),
                            ('categorical', cat_pipe, cat_feat)
                        ])

In [37]:
model_3 = Pipeline([
    ('ct', ct),
    ('classifier', DecisionTreeClassifier())
])

In [38]:
# model_3.fit(X_train, y_train)
# print(model_3.score(X_train, y_train))

In [40]:
# make_submission(model_3, X_test)
# The model scored: 0.7884

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=0, strategy='mean')),
    ('scaler', StandardScaler())
])


cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer(remainder='drop',
                        transformers=[
                            ('numerical', num_pipe, num_feat),
                            ('categorical', cat_pipe, cat_feat)
                        ])


model_4 = Pipeline([
    ('ct', ct),
    ('classifier', RandomForestClassifier(n_jobs=6))
])

In [43]:
# model_4.fit(X_train, y_train)
# print(model_4.score(X_train, y_train))

In [44]:
# make_submission(model_4, X_test)
# The model scored: 0.8189

# Dimensionality Reduction & Randomized Search

In [45]:
from sklearn.decomposition import TruncatedSVD

In [46]:
from sklearn.model_selection import RandomizedSearchCV
import scipy

In [52]:
num_pipe = Pipeline([
    ('imputer', SimpleImputer(missing_values=0, strategy='mean')),
    ('scaler', StandardScaler())
])


cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

ct = ColumnTransformer(remainder='drop',
                        transformers=[
                            ('numerical', num_pipe, num_feat),
                            ('categorical', cat_pipe, cat_feat)
                        ])

model_5 = Pipeline([
    ('ct', ct),
    ('pca', TruncatedSVD()),
    ('classifier', RandomForestClassifier())
])

param_grid = {
              'classifier__max_depth': scipy.stats.binom(9, .3, loc=1),
              'pca__n_components': range (20,50,10),
              'classifier__min_samples_split': scipy.stats.binom(90, .5, loc=10)}

model_6 = RandomizedSearchCV(model_5,
                   param_grid,
                   cv=4,
                   n_iter=100,
                   n_jobs=-1,
                   verbose=1)


In [53]:
model_6.fit(X_train, y_train)
print(model_6.score(X_train, y_train))

Fitting 4 folds for each of 100 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 12.5min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 28.4min finished


0.6167508417508417


In [54]:
make_submission(model_6, X_test)
# The model scored: 